# Thai N-NER: Thai Nested Named Entity Recognition

This demo notebook provides a tutorial on using Thai N-NER, with references from [Thai N-NER](https://medium.com/airesearch-in-th/thai-n-ner-thai-nested-named-entity-recognition-1969f8fe91f0)

Learn more about Thai N-NER here : [Thai N-NER](https://medium.com/airesearch-in-th/thai-n-ner-thai-nested-named-entity-recognition-1969f8fe91f0)

## 1. Setup and Preprocessing

In [ ]:
!pip install seqeval
!pip install pythainlp
!pip install transformers==4.29.2
!pip install sentencepiece
!pip install gdown
!pip install thai_nner
!pip install protobuf==3.20.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.16.1 requires protobuf<6.0.0dev,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.


# Model checkpoints

> Thai N-NER provides necessary resources, including models, datasets, and pre-trained language models, available here : [Thai N-NER (resources)](https://drive.google.com/drive/folders/1Dy-360iZ9hIA-xA0yizSwmpM8sx6rrjJ?usp=sharing)

To utilize this, please follow these steps::

1. Add the Shared Folder [Thai N-NER (resources)](https://drive.google.com/drive/folders/1Dy-360iZ9hIA-xA0yizSwmpM8sx6rrjJ?usp=sharing)  to Your Google Drive.
* first open the shared folder link in your web browser
* Click the folder named "thai-nner" at the top of the page.
* In the menu bar, click "Organize", then click "Add shortcut" to Drive (you may see an icon that looks like a Drive logo with a plus sign)
* Select "My Drive"


In [ ]:
# Clone github
!git clone https://github.com/vistec-AI/Thai-NNER.git
%cd /content/Thai-NNER

Cloning into 'Thai-NNER'...
remote: Enumerating objects: 274, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 274 (delta 16), reused 5 (delta 2), pack-reused 239 (from 1)
Receiving objects: 100% (274/274), 5.23 MiB | 10.03 MiB/s, done.
Resolving deltas: 100% (134/134), done.
/content/Thai-NNER


# Mount your drive to Google Colab.

In [ ]:
# Load data
from google.colab import drive
drive.mount('/content/drive/')

# Create symbolic links
!ln -s "/content/drive/MyDrive/thai-nner/lm" ./data/lm
!ln -s "/content/drive/MyDrive/thai-nner/checkpoints" ./data/checkpoints

Mounted at /content/drive/


# Inference

In [ ]:
import json
import torch
import argparse
from tqdm import tqdm
from tabulate import tabulate

from utils.unique import unique
import model.loss as module_loss
import model.model as module_arch
import model.metric as module_metric
from parse_config import ConfigParser
import data_loader.data_loaders as module_data

PAD = '<pad>'

In [ ]:
resume = 'data/checkpoints/1102_151935/checkpoint.pth'

In [ ]:
args = argparse.ArgumentParser(description='PyTorch Template')
args.add_argument('-c', '--config', default=None, type=str, help='config file path (default: None)')
args.add_argument('-r', '--resume', default=f"{resume}", type=str, help='path to latest checkpoint (default: None)')
args.add_argument('-d', '--device', default=None, type=str, help='indices of GPUs to enable (default: all)')
args.add_argument('-f', '--file', default=None, type=str, help='Error')
config = ConfigParser.from_args(args)
logger = config.get_logger('test')

# build model architecturea
model = config.init_obj('arch', module_arch)

# get function handles of loss and metrics
criterion = getattr(module_loss, config['loss'])
metric_fns = [getattr(module_metric, met) for met in config['metrics']]

logger.info('Loading checkpoint: {} ...'.format(config.resume))
checkpoint = torch.load(config.resume)
state_dict = checkpoint['state_dict']

if config['n_gpu'] > 1:
    model = torch.nn.DataParallel(model)

model.load_state_dict(state_dict)
layers_train = config._config['trainer']['layers_train']

# prepare model for testing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

total_loss = 0.0
total_metrics = torch.zeros(len(metric_fns))
# logger.info(model)

Loading checkpoint: data/checkpoints/1102_151935/checkpoint.pth ...


<ipython-input-31-31ceea9251e1>:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(config.resume)


In [ ]:
# Loading only few testing examples.
config.config['data_loader']['args']['sample_data'] = True

data_loader = config.init_obj('data_loader', module_data)
test_data_loader = data_loader.get_test()

> Now, Let's try using the pre-trained Thai N-NER model checkpoint to perform inference and predict NE tags.

In [ ]:
from utils.prediction import predict, get_dict_prediction, show



text = " วันนี้วันที่ 27 มกราคม 2568 เป็นวันที่อากาศดีมาก "


tokens, out = predict(model, text, data_loader, config)
tokens = [tk for tk in tokens if tk!=PAD]
print("|".join(tokens), "\n")
[show(x) for x in out];

<s>||วันนี้|วันที่||27||มกราคม||25|68||เป็น|วันที่||อากาศ||ดีมาก||</s> 

[2, 3]         rel            วันนี้
[3, 6]         day            วันที่27
[3, 11]        date           วันที่27มกราคม2568
[5, 6]         cardinal       27
[7, 8]         month          มกราคม
[9, 11]        year           2568


In [ ]:
text = "คณะกรรมการ 40 ปี 14 ตุลาเพื่อประชาธิปไตรสมบูรณ์"
tokens, out = predict(model, text, data_loader, config)
tokens = [tk for tk in tokens if tk!=PAD]
print("|".join(tokens), "\n")
[show(x) for x in out];

<s>||คณะกรรมการ||40||ปี||14|||ตุ|ลา|เพื่อ||ประชา||ธิ|ป||ไต|ร||สมบูรณ์|</s> 

[4, 5]         cardinal       40
[4, 7]         duration       40ปี
[6, 7]         unit           ปี
[8, 9]         day            14
[8, 13]        date           14ตุลา
[10, 13]       month          ตุลา
[14, 24]       norp_political ประชาธิปไตรสมบูรณ์


In [ ]:
text = " วันที่ 18 มกราคม 2568 เมื่อเวลา 11.15 น. ที่จ.นครพนม นายทักษิณ ชินวัตร อดีตนายกฯ ให้สัมภาษณ์กรณีนายชาดา ไทยเศรษฐ์ อดีต รมช.มหาดไทย เซ็นคำสั่งเพิกถอนที่ดินสนามกอล์ฟอัลไพน์ กลับคืนเป็นที่ธรณีสงฆ์ ก่อนหมดวาระเพียงไม่กี่วัน "
tokens, out = predict(model, text, data_loader, config)
tokens = [tk for tk in tokens if tk!=PAD]
print("|".join(tokens), "\n")
[show(x) for x in out];

<s>||วันที่||18||มกราคม||25|68||เมื่อ|เวลา||11.|15||น|.||ที่|จ|.||นคร|พ|นม||นาย||ทักษิณ|||ชิน|วั|ตร|||อดีต|นาย|ก|ฯ||ให้||สัมภาษณ์||กรณี|นาย||ชา||ดา|||ไทย||เศรษฐ|์|||อดีต|||รม|ช|.|ม|หาด|ไทย|||เซ็น||คําสั่ง||เพ|ิก|ถอน||ที่ดิน||สนาม|กอล์ฟ||อัล|ไพ|น์|||กลับ|คืน|เป็น|ที่|ธร|ณี|สงฆ์||ก่อน||หมด|วา|ระ||เพียง|ไม่||กี่|วัน||</s> 

[2, 5]         day            วันที่18
[2, 10]        date           วันที่18มกราคม2568
[4, 5]         cardinal       18
[6, 7]         month          มกราคม
[8, 10]        year           2568
[12, 19]       time           เวลา11.15น.
[14, 16]       cardinal       11.15
[14, 19]       time           11.15น.
[17, 19]       unit           น.
[21, 27]       province       จ.นครพนม
[23, 27]       province       นครพนม
[28, 29]       title          นาย
[28, 36]       person         นายทักษิณชินวัตร
[29, 31]       firstname      ทักษิณ
[32, 36]       last           ชินวัตร
[37, 42]       role           อดีตนายกฯ
[48, 49]       title          นาย
[48, 59]       person        

In [ ]:
text = " สธ.กางตัวเลขเบื้องต้นคนป่วยจากปัญหาฝุ่น PM2.5 แค่ 3 สัปดาห์ของเดือน ม.ค.พุ่ง 144,000 คนส่วนใหญ่ผิวหนัง ตาอักเสบ โรคหืด พบ 5 จังหวัดค่าฝุ่นเกิน 75 มคก.ต่อ ลบ.ม.ต่อเนื่องเกิน 3 ในระดับสีแดง "
tokens, out = predict(model, text, data_loader, config)
tokens = [tk for tk in tokens if tk!=PAD]
print("|".join(tokens), "\n")
[show(x) for x in out];

<s>||ส|ธ||.||กา|ง||ตัวเลข||เบื้องต้น|คน|ป่วย|จาก||ปัญหา||ฝุ่น||PM|2.5|||แค่||3|||สัปดาห์|ของ|เดือน||ม|.|ค|.||พุ่ง||14|4,000||คน||ส่วนใหญ่||ผิวหนัง|||ตา||อักเสบ|||โรค|ห|ืด|||พบ||5||จังหวัด|ค่า||ฝุ่น||เกิน||75||ม|ค|ก|.|ต่อ|||ลบ||.|ม|.||ต่อเนื่อง||เกิน||3||ใน||ระดับ||สีแดง||</s> 

[2, 6]         goverment      สธ.
[21, 23]       natural_disasterPM2.5
[22, 23]       cardinal       2.5
[26, 31]       duration       3สัปดาห์
[27, 28]       cardinal       3
[29, 31]       unit           สัปดาห์
[32, 38]       month          เดือนม.ค.
[34, 38]       month          ม.ค.
[41, 43]       cardinal       144,000
[41, 45]       quantity       144,000คน
[44, 45]       unit           คน
[50, 54]       disease        ตาอักเสบ
[63, 64]       cardinal       5
[63, 66]       quantity       5จังหวัด
[65, 66]       unit           จังหวัด
[72, 73]       cardinal       75
[91, 92]       cardinal       3
